In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data Pre-Processing

We need to load in the data and combine it into a merged dataframe. This will give us quite a few features as well as the labels we want. 

### Load Data

There are two datasets we are utilizing. The first are the college statistics for all college players. The second are the draft prospects for previous years.

In [2]:
college_statistics = pd.read_csv("../archive/college_statistics.csv")
draft_prospects = pd.read_csv("../archive/nfl_draft_prospects.csv")

In [3]:
# Flip the dataframe so that there is only one entry per year per individual

college_statistics_pivot = college_statistics.pivot_table(index=['player_id', 'alt_player_id', 'player_name', 'pos_abbr', 'school', 'school_abbr', 'school_primary_color', 'school_alt_color', 'season', 'active', 'all_star'],
                            columns='statistic',
                            values='value').reset_index()

In [4]:
# Really only care about the most recent year we think so only take the most recent entry according to season

idx = college_statistics_pivot.groupby('player_id')['season'].idxmax()
college_statistics_filtered = college_statistics_pivot.loc[idx]

In [5]:
threshold_value = 2014
column_to_check = 'draft_year'  # Adjust this column as needed
draft_prospects_filter = draft_prospects[(draft_prospects[column_to_check] >= threshold_value) & (draft_prospects[column_to_check] < 2021)]

In [6]:
merged_df = pd.merge(college_statistics_filtered, draft_prospects_filter, on='player_id', how='left')

In [7]:
merged_df.iloc[:,70:80]

,player_name_y,position,pos_abbr_y,school_y,school_name,school_abbr_y,link,pick,overall,round
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1121,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1122,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1123,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Feature Selection

We need to select the features that are most likely to contribute to an accurate result. To do this I simply looked through the columns and their data to get a sense of what might be interesting. The selected columns are below with rational for selection.

- 3: pos_abbr_x | This column contains the position. If we can encode this the model can recognize which features are useful for each position.
- 10: all_star | Need to encode this as 0 or 1 but this is a useful metric. Presumably if you are all star you are more likely to get drafted.
- 11 to 68 | These are all the player statistics. Assuming they are all ints or floats this can just be thrown in.

### Label Selection

We want to predict the overall draft pick so we are going to utilize the overall column

- 78 overall

In [8]:
selected_columns = [3, 10] + list(range(11, 69)) + [78]
feature_label = merged_df.iloc[:, selected_columns]

### True/False to 1/0

Column all_star has true and false values. For this column to be useful to the model it needs to be numbers not letters.

In [9]:
feature_label['all_star'] = feature_label['all_star'].astype(int)

/tmp/ipykernel_13718/3978633289.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_label['all_star'] = feature_label['all_star'].astype(int)


### NaN Value Replacement

NaN values will screw up our model. We need to make sure that they are all filled in with 0's instead. The notable exception however is that our feature NaN values will get replaced with something different. More on that below.

In [10]:
feature_label.iloc[:, 2:60] = feature_label.iloc[:, 2:60].fillna(value=0)
feature_label

,pos_abbr_x,all_star,Assist Tackles,Completion Percentage,Completions,Extra Points Made,FGM 1-19 yards,FGM 20-29 yards,FGM 30-39 yards,FGM 40-49 yards,...,Total Kicking Points,Total Points,Total Sacks,Total Tackles,Total Touchdowns,Total Two Point Conversions,Yards Per Pass Attempt,Yards Per Reception,Yards Per Rush Attempt,overall
0,DE,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,0.0,NaN
1,DE,0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,68.0,0.0,0.0,0.0,0.0,0.0,NaN
2,LB,0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,NaN
3,DE,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,NaN
4,OG,0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,137.0,0.0,0.0,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121,DE,0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,13.0,0.0,0.0,0.0,0.0,0.0,NaN
1122,WR,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,0.0,NaN
1123,PK,0,0.0,0.0,0.0,12.0,0.0,1.0,0.0,1.0,...,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
1124,DE,0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,29.0,0.0,0.0,0.0,0.0,0.0,NaN


For the labels, we don't want the NaN value to be 0... this would imply that these players are actually the best of the best! But if we instead put the value to be something like inf we will basically make it so that our model will always predict something not in the 1-200 range (because inf is such a crazy weight). Instead we're going to modify this column that so that any NaN values will be filled in with 0 and all other values (those that got drafted) will be a 1. 

This changes our model from a regression model, to a classification one. Might as well change the title of the column too...

In [11]:
mask = feature_label['overall'].notna()

# Fill non-NaN values with a specific value
feature_label.loc[mask, 'overall'] = 1
feature_label['overall'] = feature_label.iloc[:, -1].fillna(value=0)

/tmp/ipykernel_13718/1556861750.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_label['overall'] = feature_label.iloc[:, -1].fillna(value=0)


In [12]:
# Change the title of the column

feature_label = feature_label.rename(columns={'overall':'Drafted?'})

### One-hot Encoding

The position column contains 19 different positions. Again, we can't have anything with letters in it. Gotta convert it. One might think that we can simply replace with different values such as 1, 2, 3, etc. However, this places priority and weights on different positions... we don't want that! Instead we utilize one-hot encoding.

In [13]:
one_hot_encoded = pd.get_dummies(feature_label['pos_abbr_x'], prefix='encoded_column')
clean_data = pd.concat([feature_label, one_hot_encoded], axis=1).drop(columns=['pos_abbr_x'])
clean_data = clean_data.replace({False: 0, True: 1})

In [14]:
clean_data

,all_star,Assist Tackles,Completion Percentage,Completions,Extra Points Made,FGM 1-19 yards,FGM 20-29 yards,FGM 30-39 yards,FGM 40-49 yards,FGM 50+ yards,...,encoded_column_OG,encoded_column_OLB,encoded_column_OT,encoded_column_P,encoded_column_PK,encoded_column_QB,encoded_column_RB,encoded_column_S,encoded_column_TE,encoded_column_WR
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,0,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121,0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1122,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,1
1123,0,0.0,0.0,0.0,12.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,1,0,0,0,0,0
1124,0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
labels = clean_data['Drafted?']
features = clean_data.drop(columns=['Drafted?'])

# Model Building

First one we're going to do is a Basic Neural network with Tensorflow.

Others in the project are splitting up the data based on roles to make their predictions better. I am instead using their position as another feature and hoping the neural network recognizes the relationships between the role and the features that are important to getting drafted

In [16]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

2024-03-28 13:17:49.602052: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-28 13:17:50.058323: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-28 13:17:50.060807: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-28 13:17:52.206565: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2)

In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

2024-03-28 13:17:55.951063: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-28 13:17:56.273665: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/10
23/23 [==============================] - 1s 18ms/step - loss: 4.0669 - accuracy: 0.5181 - val_loss: 1.8054 - val_accuracy: 0.5111
Epoch 2/10
23/23 [==============================] - 0s 3ms/step - loss: 1.5902 - accuracy: 0.5292 - val_loss: 2.2028 - val_accuracy: 0.5833
Epoch 3/10
23/23 [==============================] - 0s 8ms/step - loss: 1.4477 - accuracy: 0.5833 - val_loss: 2.2381 - val_accuracy: 0.5944
Epoch 4/10
23/23 [==============================] - 0s 6ms/step - loss: 1.7899 - accuracy: 0.5694 - val_loss: 1.3154 - val_accuracy: 0.5278
Epoch 5/10
23/23 [==============================] - 0s 2ms/step - loss: 1.0348 - accuracy: 0.5750 - val_loss: 2.7881 - val_accuracy: 0.5667
Epoch 6/10
23/23 [==============================] - 0s 2ms/step - loss: 1.3516 - accuracy: 0.6181 - val_loss: 1.2160 - val_accuracy: 0.5778
Epoch 7/10
23/23 [==============================] - 0s 2ms/step - loss: 0.9237 - accuracy: 0.6181 - val_loss: 1.2399 - val_accuracy: 0.6000
Epoch 8/10
23/23 [=

In [19]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

predictions = model.predict(X_test)

8/8 [==============================] - 0s 2ms/step - loss: 1.1532 - accuracy: 0.5708
Test Loss: 1.1532127857208252, Test Accuracy: 0.5707964897155762
8/8 [==============================] - 0s 945us/step


Let's be real, it doesn't really seem to perform any better than a coin flip. Let's try some more basic models

In [22]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X_train, y_train)
clf.score(X_test, y_test)

/home/matteo/anaconda3/envs/tensorflow_practice/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.6061946902654868

Logistic doesn't seem to do too much better either